In [1]:
import json
from joblib import load, dump
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

In [2]:
data = pd.read_csv("data/tickets_inputs_eng_2.csv")

In [3]:
data.describe()

,complaint_what_happened,ticket_classification,processed_text,relevant_topics
count,18961,18961,18961,18961
unique,18822,78,18714,3
top,Chase has violated 15 USC 1692 by continuing c...,Credit card or prepaid card + General-purpose ...,chase continu collect activ report complet cre...,Mortgage/Loan
freq,11,4918,12,16376


In [4]:
data.head(2)

,complaint_what_happened,ticket_classification,processed_text,relevant_topics
0,Good morning my name is XXXX XXXX and I apprec...,Debt collection + Credit card debt,morn name appreci chase bank cardmemb servic c...,Mortgage/Loan
1,I upgraded my XXXX XXXX card in XX/XX/2018 and...,Credit card or prepaid card + General-purpose ...,card anniversari date inform order account ann...,Mortgage/Loan


In [5]:
data["relevant_topics"].value_counts()

relevant_topics
Mortgage/Loan                    16376
Bank Account Services             2358
Credit Report or Prepaid Card      227
Name: count, dtype: int64

In [6]:
X_text = data["processed_text"]
y_encoded = data["relevant_topics"]

In [7]:
def read_idx2label(json_path: str) -> pd.Series:
    """This function read the json file and return a dictionary
    Args:
      json_path (str): path to the json file
     Returns:
      idx2label (dict): dictionary with the mapping"""
    with open(json_path) as f:
        idx2label = json.load(f)
    return idx2label

idx2label = read_idx2label(json_path="../topic_mapping_1.json")

In [8]:
def decode_labels_into_idx(labels: pd.Series, idx2label: dict) -> pd.Series:
    """This function decode the labels into idx
    Args:
      labels (pd.Series): series with the labels
      idx2label (dict): dictionary with the mapping
     Returns:
      labels (pd.Series): series with the labels decoded
    """
    return labels.map(idx2label)

In [9]:
label2idx = {value: key for key, value in idx2label.items()}
y = decode_labels_into_idx(labels=y_encoded, idx2label=label2idx)

In [10]:
y

0        2
1        2
2        2
3        0
4        2
        ..
18956    2
18957    2
18958    2
18959    2
18960    2
Name: relevant_topics, Length: 18961, dtype: object

In [11]:
vectorizer =TfidfVectorizer()
X = vectorizer.fit_transform(X_text)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

In [13]:
majority_class = y_train.value_counts().idxmax()
print(majority_class)

2


In [14]:
dummy_classifer = DummyClassifier(strategy="constant", constant=majority_class)
dummy_classifer.fit(X_train, y_train)

DummyClassifier(constant='2', strategy='constant')

In [15]:
baseline_predictions = dummy_classifer.predict(X_test)
print(classification_report(y_test, baseline_predictions))

/home/jam/anaconda3/envs/mlops/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jam/anaconda3/envs/mlops/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       705
           1       0.00      0.00      0.00        79
           2       0.86      1.00      0.93      4905

    accuracy                           0.86      5689
   macro avg       0.29      0.33      0.31      5689
weighted avg       0.74      0.86      0.80      5689



/home/jam/anaconda3/envs/mlops/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
unique_classes, counts = np.unique(baseline_predictions, return_counts=True)

for label, count in zip(unique_classes, counts):
    print(f'Clase: {label}, Cantidad de predicciones: {count}')

Clase: 2, Cantidad de predicciones: 5689


Los datos de predicción muestran una distribución en la que la clase 2 tiene una cantidad significativamente mayor de predicciones en comparación con las otras clases. Mientras tanto, en los datos reales de prueba (y_test), se observa una distribución más equilibrada entre las clases.

Esta discrepancia entre las predicciones del modelo baseline y los datos reales subraya la simplicidad del modelo baseline y su incapacidad para capturar la complejidad del problema de clasificación. Es importante tener en cuenta que este modelo simple se usa principalmente como punto de referencia inicial y no refleja el rendimiento real de un modelo de clasificación más sofisticado. En problemas reales, se esperaría que un modelo más avanzado mejore significativamente estos resultados.

Si necesitas un modelo más preciso, es recomendable explorar modelos de clasificación más complejos y técnicas de ajuste de hiperparámetros o incluso modelos de aprendizaje profundo, dependiendo de la naturaleza y la complejidad de tus datos

In [17]:
# Crear la representación TF-IDF del texto  - da más importancia a palabras con mayor peso
from sklearn import svm
tfidf_vectorizer = TfidfVectorizer()

X_train, X_test, y_train, y_test = train_test_split(X_text, y, test_size=0.2, random_state=42)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Inicializar y entrenar el modelo SVM
svm_classifier = svm.SVC(kernel='linear', class_weight='balanced', random_state=42)
svm_classifier.fit(X_train_tfidf, y_train)

# Realizar predicciones en el conjunto de prueba y train 
predictions_test = svm_classifier.predict(X_test_tfidf)
predictions_train = svm_classifier.predict(X_train_tfidf)

# Medir la precisión del modelo
accuracy = accuracy_score(y_test, predictions_test)
print(f"Precisión del modelo: {accuracy:.2f}")

# Ver el reporte de clasificación
print(classification_report(y_test, predictions_test))

Precisión del modelo: 0.64
              precision    recall  f1-score   support

           0       0.18      0.46      0.26       466
           1       0.10      0.22      0.13        54
           2       0.89      0.68      0.77      3273

    accuracy                           0.64      3793
   macro avg       0.39      0.45      0.39      3793
weighted avg       0.79      0.64      0.70      3793



In [18]:
#Contamos la cantidad de etiquetas que resultaron de la predicción

unique_classes, counts = np.unique(predictions_test, return_counts=True)

for label, count in zip(unique_classes, counts):
    print(f'Clase: {label}, Cantidad de predicciones: {count}')

Clase: 0, Cantidad de predicciones: 1177
Clase: 1, Cantidad de predicciones: 126
Clase: 2, Cantidad de predicciones: 2490


In [19]:
# miramos la cantidad verdadera de etiquetas

y_test.value_counts()

relevant_topics
2    3273
0     466
1      54
Name: count, dtype: int64

Los datos de predicción muestran una distribución en la que la clase 2 tiene una cantidad significativamente mayor de predicciones en comparación con las otras clases. Mientras tanto, en los datos reales de prueba (y_test), se observa una distribución más equilibrada entre las clases.

Esta discrepancia entre las predicciones del modelo baseline y los datos reales subraya la simplicidad del modelo baseline y su incapacidad para capturar la complejidad del problema de clasificación. Es importante tener en cuenta que este modelo simple se usa principalmente como punto de referencia inicial y no refleja el rendimiento real de un modelo de clasificación más sofisticado. En problemas reales, se esperaría que un modelo más avanzado mejore significativamente estos resultados.

Si necesitas un modelo más preciso, es recomendable explorar modelos de clasificación más complejos y técnicas de ajuste de hiperparámetros o incluso modelos de aprendizaje profundo, dependiendo de la naturaleza y la complejidad de tus datos

## Dump model and reference data

Guardar set val como referencia en parquet y modelo

In [20]:
with open ('./model/svm.bin', 'wb') as f_out:
    dump(svm_classifier, f_out)

In [21]:
#Generamos un conjunto de entrenamiento a partir de los datos en X_train
train_data = pd.DataFrame({
    'texto' : X_train,
    'label' : y_train,
    'features' : X_train_tfidf,
    'predictions' : predictions_train
})

#Para evitar generar una matriz dispersa grandísima debido a que por cada dato se tenga una columna nueva hacemos:
#Generamos pesos aleatorios para la suma ponderarda (número de columnas en la matriz dispersa)
num_features = train_data['features'].iloc[0].shape[1]
weights = np.random.rand(num_features)

#Calcular la suma ponderada para cada fila en 'features y agregarla como una nueva columna
weighted_sum = train_data['features'].apply(lambda x: np.sum(x.multiply(weights)))
train_data['weighted_sum'] = weighted_sum

In [22]:
#conjunto de entrenamiento
train_data

,texto,label,features,predictions,weighted_sum
7401,origin account chase debtor skill receiv paym...,2,"(0, 187)\t0.11892319179943166\n (0, 1826)\t...",2,1.457271
13797,call chase refin home cash told cash chase cr...,2,"(0, 634)\t0.42756222346968964\n (0, 959)\t0...",2,2.633191
12609,shortsal jp chase agent receiv email state of...,2,"(0, 379)\t0.21675049976697044\n (0, 511)\t0...",2,1.986625
13649,issu judgment pertain case number issu sometim...,2,"(0, 165)\t0.15845502466661301\n (0, 187)\t0...",2,3.800358
7664,complaint chase account cfpb mention compani d...,2,"(0, 181)\t0.11656510770087931\n (0, 187)\t0...",0,2.626539
...,...,...,...,...,...
11284,worth custom bank relationship year addit year...,2,"(0, 169)\t0.013724364396594406\n (0, 187)\t...",0,5.768966
11964,regard person scam fund chase bank account cla...,2,"(0, 165)\t0.02422225478798329\n (0, 181)\t0...",2,5.564118
5390,travel account contact bank sever time cost lo...,2,"(0, 129)\t0.15688445162376038\n (0, 187)\t0...",2,3.115874
860,letter fr om chase state th credit card intere...,2,"(0, 1599)\t0.2345680430255364\n (0, 1826)\t...",2,1.145125


In [23]:
#Generamos un conjunto de entrenamiento a partir de los datos en X_test
test_data = pd.DataFrame({
    'texto' : X_test,
    'label' : y_test,
    'features' : X_test_tfidf,
    'predictions' : predictions_test
})

#Para evitar generar una matriz dispersa grandísima debido a que por cada dato se tenga una columna nueva hacemos:
#Generamos pesos aleatorios para la suma ponderarda (número de columnas en la matriz dispersa)
num_features = test_data['features'].iloc[0].shape[1]
weights = np.random.rand(num_features)

#Calcular la suma ponderada para cada fila en 'features y agregarla como una nueva columna
weighted_sum = test_data['features'].apply(lambda x: np.sum(x.multiply(weights)))
test_data['weighted_sum'] = weighted_sum

In [24]:
test_data

,texto,label,features,predictions,weighted_sum
10644,mail jpchase research department/archives/prop...,2,"(0, 261)\t0.08169304670272645\n (0, 511)\t0...",0,2.650980
17303,fee refund refin attempt bank zip code.th fee ...,2,"(0, 803)\t0.10596872378972176\n (0, 943)\t0...",2,2.688520
8250,car drunk driver park middl night insur compan...,2,"(0, 187)\t0.028881131580168303\n (0, 511)\t...",2,2.305398
16341,2018 ct card servic supervisor telephon center...,0,"(0, 47)\t0.2687892916887694\n (0, 324)\t0.1...",0,2.027004
14016,car total co ck bank need process gap claim wa...,0,"(0, 241)\t0.2963011233165569\n (0, 959)\t0....",2,2.261423
...,...,...,...,...,...
7687,fyi forc chang fix issu compani credit card em...,2,"(0, 187)\t0.03555354721199063\n (0, 247)\t0...",2,2.900336
9361,bank account yield account bank husband proces...,2,"(0, 169)\t0.09092833400303228\n (0, 187)\t0...",0,2.623816
18397,year histori credit score work hour week mont...,2,"(0, 187)\t0.09121649576362557\n (0, 273)\t0...",2,3.981047
4132,year stop call,2,"(0, 1537)\t0.37867560654113414\n (0, 10271)...",2,1.014344


# Data quality

In [25]:
from evidently import ColumnMapping
from evidently.report import Report
from evidently.metrics import ColumnDriftMetric, DatasetDriftMetric, DatasetMissingValuesMetric

In [26]:
#hacemos unas tranformaciones para que la librería evidently tenga los valores correctamente con su tipado
test_data["label"] = test_data["label"].astype(int)
train_data["label"] = train_data["label"].astype(int)

#aseguramos que las predicciones sean int y no object
test_data["predictions"] = test_data["predictions"].astype(int)
train_data["predictions"] = train_data["predictions"].astype(int)

In [27]:
train_data.head(2)

,texto,label,features,predictions,weighted_sum
7401,origin account chase debtor skill receiv paym...,2,"(0, 187)\t0.11892319179943166\n (0, 1826)\t...",2,1.457271
13797,call chase refin home cash told cash chase cr...,2,"(0, 634)\t0.42756222346968964\n (0, 959)\t0...",2,2.633191


In [28]:
#transformamos la parte de las features
#la columnas features es de tipo object, necesitamos que sea un toarray
#es decir indicarle que esa columna es verdaderamente una matriz dispersa
#generamos columna nueva

test_data["features_dense"] = test_data["features"].apply(lambda x: x.toarray())
train_data["features_dense"] = train_data["features"].apply(lambda x: x.toarray())

In [29]:
#Revisamos que se haya hecho la transformación
train_data.sample(2)

,texto,label,features,predictions,weighted_sum,features_dense
3645,appli approv amazon reward visa card color iss...,2,"(0, 187)\t0.1261376544233375\n (0, 494)\t0....",2,2.701884,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
5182,2019 deposit check jp chase texa payrol check ...,0,"(0, 49)\t0.09448444566740115\n (0, 187)\t0....",0,3.002906,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [30]:
#Hay algunas columnas que no son necesarias para evidently 
#todo lo que se va a anlaizar por evidently debe estar en formato hasheable
#Es decir que se puedar iterar
#Vamos a hacer una agrupación, dado test y train solamente contengan las columnas bajo el formato evidently

test_data = test_data[["label", "predictions", "weighted_sum"]]
train_data = train_data[["label", "predictions", "weighted_sum"]]


In [31]:
train_data.sample(2)

,label,predictions,weighted_sum
8028,2,2,1.716825
7171,2,2,2.047902
